# MTP TASK LIST:
https://docs.google.com/document/d/1fU2ct53ImvOxZRE9IEZgA4B1L2nCg_MBlY4LIVGeLgM/edit

# GitHub Repository:
https://github.com/IshanRD19/Master_Thesis_Project

In [31]:
# from google.colab import drive
# drive.mount('/content/drive')

# Import the required libraries

In [32]:
# Provides functions to interact with the file system
import os

# Fast and flexible means for data manipulation and analysis
import pandas as pd

# An interface for hashing any raw message in an encrypted format.
import hashlib as hl


# Read the text files 

In [33]:
def scan_and_return_text(filename):
  '''
  To scan the input file and return the text
  '''
  with open(filename) as f:
    lines_of_code = f.readlines()

  return lines_of_code


def display_code(lines_of_code):
  '''
  To display the lines of code
  '''
  for line in lines_of_code:
    print(line, end='')


# Remove irrelevant features

In [34]:
def concatenate_code(lines_of_code):
  '''
  To concatenate the lines of code and return a string
  '''
  
  return ''.join(lines_of_code)


def get_clean_code(code_as_text):
  '''
  To clean the code by removing newline characters and whitespaces
  '''
  code_as_text = code_as_text.replace('\n', '')
  code_as_text = ''.join(code_as_text.split())
  return code_as_text


def preprocess_code(lines_of_code):

  # Combine the different lines of code to form a text
  code_as_text = concatenate_code(lines_of_code)

  # Remove newline characters and whitespaces from the text
  clean_code = get_clean_code(code_as_text)
  
  return clean_code



# Derive k-grams from text

In [35]:
def derive_k_grams(clean_code, k=5):
  '''
  To derive the sequence of k-grams from the text 
  '''
  length = len(clean_code)
  k_grams = []

  for i in range(length - k + 1):
    k_grams.append(clean_code[i:i+k])

  return k_grams

# Generate hashes of k-grams

In [36]:
def generate_hash(k_gram):
  '''
  To generate the equivalent 40-bit hexadecimal code
  '''
  encoded_text = k_gram.encode('utf-8')
  hash_value = hl.sha1(encoded_text)
  return hash_value.hexdigest()


def fetch_hash_values(k_grams):
  '''
  To return the hash values for each given k-gram
  '''
  hash_values = []

  for k_gram in k_grams:
    hash_values.append(generate_hash(k_gram))

  return hash_values

# Implement Winnowing Algorithm

In [37]:
def extract_windows(hash_values, window_size):
  '''
  To extract windows of the given size
  '''
  windows = []

  for i in range(len(hash_values) - window_size + 1):
    windows.append(hash_values[i:i+window_size])

  return windows


def implement_winnowing(windows, window_size):
  '''
  To select a fingerprint from each window
  '''
  w = 0
  fingerprints = []
  window_count = len(windows)

  # Traverse through all the windows
  while w < window_count:
    min_hash_value, min_hash_value_index = windows[w][0], 0

    # For each hash value in the window
    for i in range(1, window_size):

      # Compare and store the minimum hash value
      if windows[w][i] < min_hash_value:
        min_hash_value = windows[w][i]
        min_hash_value_index = i

    fingerprints.append(min_hash_value)
    w = w + min_hash_value_index + 1

  return fingerprints


hash_values = [77, 74, 42, 17, 98, 50, 17, 98, 8, 88, 67, 39, 77, 74, 42, 17, 98]
windows = extract_windows(hash_values, 4)
windows

[[77, 74, 42, 17],
 [74, 42, 17, 98],
 [42, 17, 98, 50],
 [17, 98, 50, 17],
 [98, 50, 17, 98],
 [50, 17, 98, 8],
 [17, 98, 8, 88],
 [98, 8, 88, 67],
 [8, 88, 67, 39],
 [88, 67, 39, 77],
 [67, 39, 77, 74],
 [39, 77, 74, 42],
 [77, 74, 42, 17],
 [74, 42, 17, 98]]

In [38]:
fingerprints = implement_winnowing(windows, 4)
fingerprints

[17, 17, 8, 39, 17]

# Check for Plagiarism

In [39]:
def check_for_plagiarism(path, filename_1, filename_2, verbose=False):
  '''
  To compare given codes for plagiarism
  '''
  # Read the two text files
  code_1 = scan_and_return_text(path + filename_1)
  code_2 = scan_and_return_text(path + filename_2)

  # Display the codes
  # display_code(code_1)
  # display_code(code_2)

  # Pre-process the codes
  clean_code_1 = preprocess_code(code_1)
  clean_code_2 = preprocess_code(code_2)

  # Set the noise threshold
  k = 5

  # Form k-grams from the pre-processed text
  k_grams_1 = derive_k_grams(clean_code_1, k)
  k_grams_2 = derive_k_grams(clean_code_2, k)

  # Generate hash values from the k-grams
  hash_values_1 = fetch_hash_values(k_grams_1)
  hash_values_2 = fetch_hash_values(k_grams_2)

  # Set the window size
  window_size = 4

  # Form the windows for the above hash values
  windows_1 = extract_windows(hash_values_1, window_size)
  windows_2 = extract_windows(hash_values_2, window_size)

  # Extract the fingerprints of each code
  fingerprints_1 = implement_winnowing(windows_1, window_size)
  fingerprints_2 = implement_winnowing(windows_2, window_size)

  fingerprints_of_1_in_2 = len([fingerprint for fingerprint in fingerprints_1 if fingerprint in fingerprints_2])
  fingerprints_of_2_in_1 = len([fingerprint for fingerprint in fingerprints_2 if fingerprint in fingerprints_1])

  result = round(100 * (fingerprints_of_1_in_2 + fingerprints_of_2_in_1) / (len(fingerprints_1) + len(fingerprints_2)),2)

  # Display the Plagiarism Report
  if verbose:

    print('Total Fingerprints in {}: {}'.format(filename_1, len(fingerprints_1)))
    print('Total Unique Fingerprints in {}: {}'.format(filename_1, len(set(fingerprints_1))))
    print('Total Fingerprints in {}: {}'.format(filename_2, len(fingerprints_2)))
    print('Total Unique Fingerprints in {}: {}'.format(filename_2, len(set(fingerprints_2))))

    print('----------------------------------------------------')
    print('Code {} Fingerprints present in Code {}: {}'.format(1, 2, fingerprints_of_1_in_2))
    print('Code {} Fingerprints present in Code {}: {}'.format(2, 1, fingerprints_of_2_in_1))
    print('Total Common Unique Fingerprints: {}'.format(len(list(set(fingerprints_1) & set(fingerprints_2)))))
    print('----------------------------------------------------')
    print('Plagiarism Report: {}% Match'.format(result))
    print('----------------------------------------------------')

  return result


# Run MOSS for given batch of files

In [40]:
def trigger_moss(path, want_exhaustive_logs=False, verbose=False):
  '''
  To run MOSS for all the files present in the given path 
  '''
  data = []

  # Extract all files present in the given path
  filenames = os.listdir(path)
  file_count = len(filenames)

  # For each file in directory
  for i in range(file_count):

    # Compare each file as source code
    for j in range(i+1, file_count):

      # Evaluate the files for plagiarism
      plagiarism_percentage = check_for_plagiarism(path, filenames[i], filenames[j])

      # Store the plagiarism percentage
      data.append([filenames[i], filenames[j], plagiarism_percentage])
      data.append([filenames[j], filenames[i], plagiarism_percentage])

  # Create a dataframe of acquired results
  plagiarism_logs = pd.DataFrame(data, columns=['Submitted Code', 'Source Code', 'Plagiarism (%)'])

  # Display the exhaustive list of logs
  if verbose:
    plagiarism_logs.sort_values(by=['Plagiarism (%)'], ascending=False).reset_index(drop=True)

  # Check if list of exhaustive logs is required
  if want_exhaustive_logs:
    return plagiarism_logs.sort_values(by=['Plagiarism (%)'], ascending=False).reset_index(drop=True)

  return plagiarism_logs.loc[plagiarism_logs.groupby(['Submitted Code'], sort=True)['Plagiarism (%)'].idxmax()].sort_values(by=['Plagiarism (%)'], ascending=False).reset_index(drop=True)


# Display the MOSS Report

In [41]:
TEST_PATH = "/content/drive/MyDrive/Master_Thesis_Project/tests/"

SAMPLE_TEST = TEST_PATH + 'sample_codes/'
FACTORIAL_TEST = TEST_PATH + 'factorial/'
LINKED_LIST_TEST = TEST_PATH + 'linked_list/'


In [42]:
trigger_moss(SAMPLE_TEST)

,Submitted Code,Source Code,Plagiarism (%)
0,code1.py,code2.py,84.55
1,code2.py,code1.py,84.55


In [43]:
trigger_moss(FACTORIAL_TEST)


,Submitted Code,Source Code,Plagiarism (%)
0,fact1.txt,fact4.txt,93.55
1,fact4.txt,fact1.txt,93.55
2,fact5.txt,fact4.txt,72.65
3,fact3.txt,fact1.txt,71.88
4,fact2.txt,fact1.txt,66.41


In [44]:
trigger_moss(LINKED_LIST_TEST)


,Submitted Code,Source Code,Plagiarism (%)
0,linked_list_1.txt,linked_list_3.txt,68.98
1,linked_list_3.txt,linked_list_1.txt,68.98
2,linked_list_2.txt,linked_list_3.txt,44.31
3,linked_list_4.txt,linked_list_2.txt,26.72


# Display the Originality Report

In [ ]:
# Min vs Avg of all???

In [24]:
plagiarism_logs = trigger_moss(LINKED_LIST_TEST, True)

# Display the important results
plagiarism_logs.groupby(['Submitted Code'], sort=True)['Plagiarism (%)'].mean()
# .sort_values(by=['Plagiarism (%)']).reset_index(drop=True)

Submitted Code
linked_list_1.txt    43.826667
linked_list_2.txt    38.240000
linked_list_3.txt    43.596667
linked_list_4.txt    21.010000
Name: Plagiarism (%), dtype: float64

In [ ]:
# Read the two text files
code_1 = scan_and_return_text('fact1.txt')
code_2 = scan_and_return_text('fact2.txt')

# Display the codes
# display_code(code_1)
# display_code(code_2)

# Pre-process the codes
clean_code_1 = preprocess_code(code_1)
clean_code_2 = preprocess_code(code_2)


In [ ]:
clean_code_1

'Python3programtofind#factorialofgivennumberdeffactorial(n):#singlelinetofindfactorialreturn1if(n==1orn==0)elsen*factorial(n-1);#DriverCodenum=5;print("Factorialof",num,"is",factorial(num))'

In [ ]:
clean_code_2

'#Python3programtofind#factorialofgivennumberdeffactorial(n):ifn<0:return0elifn==0orn==1:return1else:fact=1while(n>1):fact*=nn-=1returnfact#DriverCodenum=5;print("Factorialof",num,"is",factorial(num))'

In [ ]:
# Previous Results: 
# ------------------------------------------------------------

# <<< On removing newline and whitespace >>>

# Total Fingerprints in Code 1: 397
# Total Unique Fingerprints in Code 1: 184
# Total Fingerprints in Code 2: 276
# Total Unique Fingerprints in Code 2: 122
# ----------------------------------------------------
# Code 1 Fingerprints present in Code 2: 293
# Code 2 Fingerprints present in Code 1: 276
# Total Common Unique Fingerprints: 122
# ----------------------------------------------------
# Plagiarism Report: 84.55% Match

# ------------------------------------------------------------

# <<< On Raw Code >>>

# Total Fingerprints in Code 1: 749
# Total Unique Fingerprints in Code 1: 215
# Total Fingerprints in Code 2: 606
# Total Unique Fingerprints in Code 2: 149
# ----------------------------------------------------
# Code 1 Fingerprints present in Code 2: 638
# Code 2 Fingerprints present in Code 1: 606
# Total Common Unique Fingerprints: 149
# ----------------------------------------------------
# Plagiarism Report: 91.81% Match
# ------------------------------------------------------------

In [ ]:
code_1

['import math\n',
 'def dist2(p1,p2):\n',
 '    d=math.sqrt(((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2))\n',
 '    return d\n',
 'def dist3(p1,p2,p3):\n',
 '    d1=math.sqrt(((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2))\n',
 '    d2=math.sqrt(((p1[0]-p3[0])**2 + (p1[1]-p3[1])**2))\n',
 '    d=d1+d2\n',
 '    return d\n',
 'def largest_rectangle():\n',
 '    l1=[]\n',
 '    l2=[]\n',
 '    for i in range(n):\n',
 '        for j in range(i, n):\n',
 '            for k in range(j,n):\n',
 '                if points[i][0]==points[j][0] and points[i][1]==points[k][1]:\n',
 '                    x1=points[k][0]\n',
 '                    y1=points[j][1]\n',
 '                    if (x1,y1) in points:\n',
 '                        d=dist3(points[i], points[j], points[k])\n',
 '                        l1=[points[i], points[j], points[k], (x1,y1), d]\n',
 '                        l2.append(l1)\n',
 '    l2=sorted(l2, key=lambda x:x[-1], reverse=True)\n',
 '    print(l2[0])\n',
 'def largest_square():\n',
 '  

In [ ]:
clean_code_1

"importmathdefdist2(p1,p2):d=math.sqrt(((p1[0]-p2[0])**2+(p1[1]-p2[1])**2))returnddefdist3(p1,p2,p3):d1=math.sqrt(((p1[0]-p2[0])**2+(p1[1]-p2[1])**2))d2=math.sqrt(((p1[0]-p3[0])**2+(p1[1]-p3[1])**2))d=d1+d2returnddeflargest_rectangle():l1=[]l2=[]foriinrange(n):forjinrange(i,n):forkinrange(j,n):ifpoints[i][0]==points[j][0]andpoints[i][1]==points[k][1]:x1=points[k][0]y1=points[j][1]if(x1,y1)inpoints:d=dist3(points[i],points[j],points[k])l1=[points[i],points[j],points[k],(x1,y1),d]l2.append(l1)l2=sorted(l2,key=lambdax:x[-1],reverse=True)print(l2[0])deflargest_square():l1=[]l2=[]foriinrange(n):forjinrange(i,n):forkinrange(j,n):ifpoints[i][0]==points[j][0]andpoints[i][1]==points[k][1]anddist2(points[i],points[j])==dist2(points[i],points[k]):x1=points[k][0]y1=points[j][1]if(x1,y1)inpoints:d=dist3(points[i],points[j],points[k])l1=[points[i],points[j],points[k],(x1,y1),d]l2.append(l1)l2=sorted(l2,key=lambdax:x[-1],reverse=True)ifl2[0][-1]==0:print('Nosquare.')else:print(l2[0])n=int(input())poi

In [ ]:
xx = clean_code_1.split('==')
for i in xx:
  print(i.split('='))

['importmathdefdist2(p1,p2):d', 'math.sqrt(((p1[0]-p2[0])**2+(p1[1]-p2[1])**2))returnddefdist3(p1,p2,p3):d1', 'math.sqrt(((p1[0]-p2[0])**2+(p1[1]-p2[1])**2))d2', 'math.sqrt(((p1[0]-p3[0])**2+(p1[1]-p3[1])**2))d', 'd1+d2returnddeflargest_rectangle():l1', '[]l2', '[]foriinrange(n):forjinrange(i,n):forkinrange(j,n):ifpoints[i][0]']
['points[j][0]andpoints[i][1]']
['points[k][1]:x1', 'points[k][0]y1', 'points[j][1]if(x1,y1)inpoints:d', 'dist3(points[i],points[j],points[k])l1', '[points[i],points[j],points[k],(x1,y1),d]l2.append(l1)l2', 'sorted(l2,key', 'lambdax:x[-1],reverse', 'True)print(l2[0])deflargest_square():l1', '[]l2', '[]foriinrange(n):forjinrange(i,n):forkinrange(j,n):ifpoints[i][0]']
['points[j][0]andpoints[i][1]']
['points[k][1]anddist2(points[i],points[j])']
['dist2(points[i],points[k]):x1', 'points[k][0]y1', 'points[j][1]if(x1,y1)inpoints:d', 'dist3(points[i],points[j],points[k])l1', '[points[i],points[j],points[k],(x1,y1),d]l2.append(l1)l2', 'sorted(l2,key', 'lambdax:x[-1],r

In [ ]:
def generate_hash_values_using_sha_1_encoding(content):
  '''
  Generate hash values using SHA-1 encoding
  '''
  encoded_text = content.encode('utf-8')
  print(encoded_text)

  hash_text = hl.sha1(encoded_text)
  print(hash_text)
  
  # Generate the equivalent 40-bit hexadecimal code
  hex_encryption = hash_text.hexdigest()
  print(hex_encryption)

  print(hex_encryption[-4:])

  print(int(hex_encryption[-4:], 16))


  return
  


generate_hash_values_using_sha_1_encoding('abaofjnafn')


b'abaofjnafn'
<sha1 HASH object @ 0x7fab4843f4b0>
cf6035a14999753e6b44ab4f2eebe6d764bd9c4d
9c4d
40013


In [ ]:
int('1', 16)

1

In [ ]:
import pygments.token
import pygments.lexers
import hashlib
from cleanUP import *

#sha-1 encoding is used to generate hash values
def hash(text):
    #this function generates hash values
    hashval = hashlib.sha1(text.encode('utf-8'))
    hashval = hashval.hexdigest()[-4 :]
    hashval = int(hashval, 16)  #using last 16 bits of sha-1 digest
    return hashval

#function to form k-grams out of the cleaned up text
def kgrams(text, k = 25):
    tokenList = list(text)
    n = len(tokenList)
    kgrams = []
    for i in range(n - k + 1):
        kgram = ''.join(tokenList[i : i + k])
        hv = hash(kgram)
        kgrams.append((kgram, hv, i, i + k))  #k-gram, its hash value, starting and ending positions are stored
        #these help in marking the plagiarized content in the original code.
    return kgrams

#function that returns the index at which minimum value of a given list (window) is located
def minIndex(arr):
    minI = 0
    minV = arr[0]
    n = len(arr)
    for i in range(n):
        if arr[i] < minV:
            minV = arr[i]
            minI = i
    return minI

#we form windows of hash values and use min-hash to limit the number of fingerprints
def fingerprints(arr, winSize = 4):
    arrLen = len(arr)
    prevMin = 0
    currMin = 0
    windows = []
    fingerprintList = []
    for i in range(arrLen - winSize):
        win = arr[i: i + winSize]  #forming windows
        windows.append(win)
        currMin = i + minIndex(win)
        if not currMin == prevMin:  #min value of window is stored only if it is not the same as min value of prev window
            fingerprintList.append(arr[currMin])  #reduces the number of fingerprints while maintaining guarantee
            prevMin = currMin  #refer to density of winnowing and guarantee threshold (Stanford paper)

    return fingerprintList

#takes k-gram list as input and returns a list of only hash values
def hashList(arr):
    HL = []
    for i in arr:
        HL.append(i[1])

    return HL

def plagiarismCheck(file1, file2):
    f1 = open(file1, "r")
    token1 = tokenize(file1)  #from cleanUP.py
    str1 = toText(token1)
    token2 = tokenize(file2)
    str2 = toText(token2)
    kGrams1 = kgrams(str1)  #stores k-grams, their hash values and positions in cleaned up text
    kGrams2 = kgrams(str2)
    HL1 = hashList(kGrams1)  #hash list derived from k-grams list
    HL2 = hashList(kGrams2)
    fpList1 = fingerprints(HL1)
    fpList2 = fingerprints(HL2)
    start = []   #to store the start values corresponding to matching fingerprints
    end = []   #to store end values
    code = f1.read()  #original code
    newCode = ""   #code with marked plagiarized content
    points = []
    for i in fpList1:
        for j in fpList2:
            if i == j:   #fingerprints match
                flag = 0
                match = HL1.index(i)   #index of matching fingerprints in hash list, k-grams list
                newStart = kGrams1[match][2]   #start position of matched k-gram in cleaned up code
                newEnd = kGrams1[match][3]   #end position
                for k in token1:
                    if k[2] == newStart:   #linking positions in cleaned up code to original code
                        startx = k[1]
                        flag = 1
                    if k[2] == newEnd:
                        endx = k[1]
                if flag == 1:
                    points.append([startx, endx])
    points.sort(key = lambda x: x[0])
    points = points[1:]
    mergedPoints = []
    mergedPoints.append(points[0])
    for i in range(1, len(points)):
        last = mergedPoints[len(mergedPoints) - 1]
        if points[i][0] >= last[0] and points[i][0] <= last[1]: #merging overlapping regions
            if points[i][1] > last[1]:
                mergedPoints = mergedPoints[: len(mergedPoints)-1]
                mergedPoints.append([last[0], points[i][1]])
            else:
                pass
        else:
            mergedPoints.append(points[i])
    newCode = code[: mergedPoints[0][0]]
    plagCount = 0
    for i in range(len(mergedPoints)):
        if mergedPoints[i][1] > mergedPoints[i][0]:
            plagCount += mergedPoints[i][1] - mergedPoints[i][0]
            newCode = newCode + '\x1b[6;30;42m' + code[mergedPoints[i][0] : mergedPoints[i][1]] + '\x1b[0m'
            if i < len(mergedPoints) - 1:
                newCode = newCode + code[mergedPoints[i][1] : mergedPoints[i+1][0]]
            else:
                newCode = newCode + code[mergedPoints[i][1] :]
    print("Approx ratio of plagiarized content in file 1: ", (plagCount/len(code)))
    print(newCode)


fn1 = input("Enter the path/name of program 1: ")
fn2 = input("Enter the path/name of program 2: ")
plagiarismCheck(fn1, fn2)


Enter the path/name of program 1: test2.py
Enter the path/name of program 2: test3.py


UnboundLocalError: ignored

In [ ]:
#thanks to Shashwat Sanket @shashwatsanket997 for suggesting the use of python difflib module
from difflib import SequenceMatcher
from cleanUP import *

def plagerised_ratio(filename1, filename2):
    tokens1 = tokenize(filename1) #(elements of cleaned up code, their position in original code, position in cleaned up code)
    file1 = toText(tokens1)  #cleaned up code - greatly increases effectiveness of plagiarism checker
    tokens2 = tokenize(filename2)
    file2 = toText(tokens2)
    SM = SequenceMatcher(None, file1, file2)
    similarity_ratio = SM.ratio()
    print(similarity_ratio)   # ratio of plagiarised content
    blocks = list(SM.get_matching_blocks()) #elements  of blocks[] - (start-file1, start-file2, length)
    blocks = blocks[: -1]
    f1 = open(filename1, "r")
    for i in blocks:
        flag = 0
        for j in range(len(tokens1)):
            if tokens1[j][2] == i[0]:  #linking start of matching block to position in cleaned up code
                start = tokens1[j][1]  #linking position in cleaned up code to position in original code file
                flag = 1
            if tokens1[j][2] == (i[0] + i[2] - 1): #linking end to cleaned up code
                end = tokens1[j][1]  #linking to original code file
                break
        if not flag == 0 and (end - start) > 100:  #printing significant blocks of plagiarized content
            #the start and end of matching blocks is linked to the original code to properly mark the plagiarized content
            f1.seek(start, 0)
            print(f1.read(end - start))

fn1 = input("Enter the path/name of program 1: ")
fn2 = input("Enter the path/name of program 2: ")
plagerised_ratio(fn1, fn2)


Enter the path/name of program 1: test2.py
Enter the path/name of program 2: test3.py
0.7905951506245408
eturn d
def largest_rectangle():
    l1=[]
    l2=[]
    for i in range(n):
        for j in range(i, n):
            for k in range(j,n):
                if points[i][0]==points[j][0] and points[i][1]==points[k][1]:
                    x1=points[k][0]
                    y1=points[j][1]
                    if (x1,y1) in points:
                        d=dist3(points[i], points[j], points[k])
                        l1=[points[i], points[j], points[k], (x1,y1), d]
                        l2.append(l1)
    l2=sorted(l2, key=lambda x:x[-1], reverse=True)
    print(l2[0])
def largest_square():
    l1=[]
    l2=[]
    for i in range(n):
        for j in range(i, n):
            for k in range(j,n):
                if points[i][0]==points[j][0] and points[i][1]==points[k][1] and dist2(points[i], points[j])==dist2(points[i], points[k]):
                    x1=points[k][0]
                